In [4]:
import pandas as pd

df = pd.read_csv('data/시도별사교육참여율(년도).csv')
df.head(), df.shape, df['지역별'].unique()


(     시점   지역별  사교육참여율
 0  2009  대도시외    68.1
 1  2009  대도시외    77.4
 2  2009  대도시외    75.6
 3  2009  대도시외    74.0
 4  2009   광역시    75.9,
 (250, 3),
 array(['대도시외', '광역시', '서울'], dtype=object))

In [5]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'사교육참여율': 'mean'})
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)

final_monthly_df.head(12), final_monthly_df.shape, final_monthly_df['지역별'].unique()

# 데이터 저장
monthly_df.to_csv('시도별사교육참여율(월별).csv', index=False)
